In [1]:
import os, sys
sys.path.insert(0, os.path.abspath(".."))

import numpy as np
from sklearn import datasets

from elastic.core.notebook.variable_snapshot import VariableSnapshotSet
from elastic.core.notebook.record_event import RecordEvent

from sklearn.neighbors import KNeighborsClassifier

from elastic.core.globals import reset_notebook_state
reset_notebook_state()

In [2]:
@RecordEvent
def cell_2():
    iris_X, iris_y = datasets.load_iris(return_X_y=True)
    
    return VariableSnapshotSet({"iris_X": iris_X, "iris_y": iris_y})

In [3]:
iris_X, iris_y = cell_2()


container created for variable  iris_X
container created for variable  iris_y


In [4]:
@RecordEvent
def cell_3():
    np.random.seed(0)

In [5]:
cell_3()

In [6]:
@RecordEvent
def cell_4():
    indices = np.random.permutation(len(iris_X))
    iris_X_train = iris_X[indices[:-10]]
    iris_y_train = iris_y[indices[:-10]]
    iris_X_test = iris_X[indices[-10:]]
    iris_y_test = iris_y[indices[-10:]]
    
    return VariableSnapshotSet({"indices": indices, "iris_X_train": iris_X_train, "iris_y_train": iris_y_train, "iris_X_test": iris_X_test, "iris_y_test": iris_y_test})

In [7]:
indices, iris_X_train, iris_y_train, iris_X_test, iris_y_test = cell_4()


container created for variable  indices
container created for variable  iris_X_train
container created for variable  iris_y_train
container created for variable  iris_X_test
container created for variable  iris_y_test


In [8]:
@RecordEvent
def cell_5():
    indices = 123
    knn = KNeighborsClassifier()
    knn.fit(iris_X_train, iris_y_train)

    knn.predict(iris_X_test)
    
    return VariableSnapshotSet({"knn": knn, "indices": indices})

In [9]:
indices, test_var = cell_5()


container created for variable  knn
container created for variable  indices


In [10]:
@RecordEvent
def import_libraries():
    import numpy as np
    from sklearn import datasets
    return VariableSnapshotSet({"np": np, "datasets": datasets})

import_libraries()


container created for variable  np
container created for variable  datasets


(<module 'numpy' from '/Users/libilly/elastic_notebook2/elastic-notebook/venv/lib/python3.8/site-packages/numpy/__init__.py'>,
 <module 'sklearn.datasets' from '/Users/libilly/elastic_notebook2/elastic-notebook/venv/lib/python3.8/site-packages/sklearn/datasets/__init__.py'>)

In [11]:
from elastic.core.globals import operation_events

for x in operation_events:
    print(x, "\n")
    print("input variable_snapshots: \n")
    for y in x.input_variable_snapshots:
        print(y, "\n")
    print("------------")

Operation with name cell_2 and id 0 started at 2022-10-16 20:31:09.678007 and ended at 2022-10-16 20:31:09.679893, 0.001886 s elapsed, 0 input variable snapshots 

input variable_snapshots: 

------------
Operation with name cell_3 and id 1 started at 2022-10-16 20:31:09.685032 and ended at 2022-10-16 20:31:09.685051, 1.9e-05 s elapsed, 0 input variable snapshots 

input variable_snapshots: 

------------
Operation with name cell_4 and id 2 started at 2022-10-16 20:31:09.692008 and ended at 2022-10-16 20:31:09.692233, 0.000225 s elapsed, 2 input variable snapshots 

input variable_snapshots: 

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2] 

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0

In [12]:
# Create a graph representation of the current state of the notebook
from elastic.core.notebook.create_graph import create_graph
graph = create_graph()

print("---------------------------")
print("Active nodes:")
for node in graph.active_nodes:
    print(node.vs.get_name(), node.vs.get_version(), node.vs.get_size())

print("---------------------------")
print("Edges:")
for edge in graph.edges:
    print("---------------------------")
    print("Edge:", edge.oe.cell_func_name, edge.oe.duration)
    print("sources")
    for node in edge.src.nodes:
        print("   ", node.vs.get_name(), node.vs.get_version(), node.vs.get_size())
    print("destinations")
    for node in edge.dst.nodes:
        print("   ", node.vs.get_name(), node.vs.get_version(), node.vs.get_size())

---------------------------
Active nodes:
iris_X 1 4928
iris_y 1 1312
iris_X_train 1 4608
iris_y_train 1 1232
iris_X_test 1 448
iris_y_test 1 192
knn 1 48
indices 2 28
np 1 48
datasets 1 48
---------------------------
Edges:
---------------------------
Edge: cell_2 0.001886
sources
destinations
    iris_X 1 4928
    iris_y 1 1312
---------------------------
Edge: cell_3 1.9e-05
sources
destinations
---------------------------
Edge: cell_4 0.000225
sources
    iris_y 1 1312
    iris_X 1 4928
destinations
    indices 1 1312
    iris_X_train 1 4608
    iris_y_train 1 1232
    iris_X_test 1 448
    iris_y_test 1 192
---------------------------
Edge: cell_5 0.001986
sources
    iris_X_test 1 448
    iris_y_train 1 1232
    iris_X_train 1 4608
destinations
    knn 1 48
    indices 2 28
---------------------------
Edge: import_libraries 1e-05
sources
destinations
    np 1 48
    datasets 1 48


In [13]:
# Use the optimizer to find nodes to migrate
from elastic.algorithm.optimizer_greedy import OptimizerGreedy
from elastic.algorithm.optimizer_exact import OptimizerExact
from elastic.algorithm.baseline import MigrateAllBaseline
from elastic.algorithm.baseline import RecomputeAllBaseline
from elastic.algorithm.baseline import RandomBaseline

opt = OptimizerExact(migration_speed_bps=100000)
graph.trim_graph(opt)

print("---------------------------")
print("Nodes to recompute:")
for node in graph.nodes_to_recompute:
    print(node.vs.get_name(), node.vs.get_version(), node.vs.get_size())
    print(node.vs)

minimum migration cost: 0.002881
---------------------------
Nodes to migrate:
indices 2 28
knn 1 48
---------------------------
Nodes to recompute:
datasets 1 48
iris_y_test 1 48
iris_X 1 48
iris_y 1 48
iris_X_train 1 48
iris_X_test 1 48
iris_y_train 1 48
np 1 48


In [14]:
# Migrate the dependency graph.
from elastic.core.io.filesystem_adapter import FilesystemAdapter
from elastic.core.io.migrate import migrate

migrate(graph, FilesystemAdapter())

Pickling iris_X
Pickling iris_y
Pickling indices
Pickling iris_X_train
Pickling iris_y_test
Pickling iris_X_test
Pickling iris_y_train
Pickling knn
Pickling indices
Pickling np
Pickling datasets
